# ✨️閃開! 人類, 交給AI來!

本筆記本主要講述如何使用LLM產生基於特定領域知識的合成資料集

- 合成資料？📃  
  簡單來講就是用生成式AI來產生的資料. (詳見[What is synthetic data?](https://mostly.ai/what-is-synthetic-data))

- 為什麼需要基於特定領域的知識來產生合成資料？🤔  
  1. 在企業內部有許多專業領域知識(domain knowledge)都是只有在該領域的專家才懂, 且這些資料大多都不容易閱讀.
  2. 透過微調讓LLM可以更貼近特定領域的應用場景, 而要微調便需要先準備好資料.

## 運行準備
- [Python](https://www.python.org/downloads/release/python-3111/)
- [Ollama](https://ollama.com/download)  
  本文主要作為示範目的, 所以就只用llama3.1-8b-q4_0的模型來跑(效果已經很不錯了🤩)  
  詳見 [How to Run LLM Models Locally with Ollama?](https://www.analyticsvidhya.com/blog/2024/07/local-llm-deployment-with-ollama/)  
    
  (如果想要使用更大的模型, 但是卻沒有足夠的硬體, 非常推薦使用 [Groq](https://groq.com/)🚀 或是 [Nvidia NIM](https://build.nvidia.com/explore/discover)🌲)

## 安裝套件、API key設定

In [2]:
# %pip install rich unstructured[md] pydantic openai datasets pandas

## 載入文本

> 範例文本來源是我將 [Fine-tune Llama 3.1 Ultra-Efficiently with Unsloth](https://huggingface.co/blog/mlabonne/sft-llama3) 翻成中文後的Markdown格式文件. (這篇寫的超讚, 有興趣的話可以讀一讀🦥)

因為在實際應用場景中, 文本的長度非常有可能遠超 LLM的上下文長度, 所以必須先對文本做分割處理.  
本文只有用 `unstructured` 做簡單的分割, 這部份不是本文的重點就不細談.

In [3]:
from rich import print
from unstructured.chunking.title import chunk_by_title
from unstructured.partition.md import partition_md

md_file = './example.md'
elements = partition_md(filename=md_file, encoding='utf-8')
chunks = chunk_by_title(elements)

In [4]:
print(chunks[0].to_dict())

{
    'type': 'CompositeElement',
    'element_id': '11495636-972a-4a66-9f73-2bd56068add6',
    'text': '使用Unsloth對Llama 3.1進行超高效微調\n\n原文連結\n\nmlabonne Maxime 
Labonne初學者指南：最新的監督微調技術\n\nLlama 3.1 
的最新發布提供了具有驚人性能的模型，縮小了封閉源代碼和開源模型之間的差距。與其使用像 GPT-4o 和 Claude 3.5 
這樣的通用、預先訓練好的語言模型，你可以根據自己的具體需求對 Llama 3.1 
進行微調，以實現更好的性能和可定制性，並降低成本。\n\n在本文中，我們將提供一份全面性的監督微調概覽。接下來，我們將
比較監督微調與提示工程，以了解何時使用監督微調是合理的，詳細介紹主要技術及其優缺點，並介紹重要概念，如LoRA超參數、
存儲格式和聊天模板。最後，我們將在實踐中實現監督微調，通過在Google Colab中使用Unsloth對Llama 3.1 
8B進行微調，達到最先進的優化效果。本文中使用的所有代碼都可以在Google Colab和LLM Course中找到。特別感謝Daniel 
Han回答我的問題。',
    'metadata': {
        'emphasized_text_contents': ['mlabonne Maxime Labonne初學者指南：最新的監督微調技術'],
        'emphasized_text_tags': ['i'],
        'file_directory': '.',
        'filename': 'example.md',
        'filetype': 'text/markdown',
        'languages': ['eng', 'nor', 'cat'],
        'last_modified': '2024-08-01T00:33:19',
        'link_texts': ['原文連結', 'Google Colab', 'LLM Course'],
        'link_urls': [
            'https://huggingface.co/blog/mlabonne/sft-llama3',
            'https://colab.research.google.com/drive/164cg_O7SV7G8kZr_JXqLd6VC7pd86-1Z#scrollTo=PoPKQjga6obN',
            'https://github.com/mlabonne/llm-course'
        ],
        'orig_elements': 
'eJzVV1tvG1UQ/isr89rE536JxAMUqQhSKBAq1KayznW9dC/BXkNLxX9n5mzW3aSY5gnRl5NkZvzN/Rvn5btValOX+nHTxNVFtWJJJxYV18pq64n0ir
pAZXDCWhOZXD2qVl0aXXSjA/t3q+DGVA+7t5uYbsYtiAhY5KZNm9jsUhhBhbjnq1tx77qEgvTGdTdtOu/irBnf3hTNmN6M687tXsfhjx6Vrevrg6vTH
rQvV6mvUdgPO/wB7levis1+3HRDbHKTpkQIE2fEnBF6RcgF5xfUrv56NKGj/vogss7XBy2Z+bnft8O4vT7IQOJl6zpX8XN6fbCEs+uDMSLAG40EiYvm
+qCkgFdml0DufMZA5uivmrFN6OmDykYqs4Ei8ux5CiaRxJk0wRhvoyfhk64s2jX96w16mMCgPtxmrJXRWEnGodZRkgkTbQ+7djLdjuPN/mK93h7quun
r7EI6D8Pat0O97lrnh75P630ez1psDV+9utvHE34e0hORhYB0oqKGy8xydERL5YPjVivj0sN6krqbrds3f6ZY8t+EoR/Bx5TcnED11L1pulRdTn9C2I
xGeL2GUTKEwHApTjSmI+HNmTqQaEIwNQ8vBCTwTZigVnw5gGDDDMFRlbwU+H5Io6uncJqi/p9O0bKv/33dHjIwPBFrA2WOeZWEzFL4pGmWmYakrGef9
BIvy39kwWqu4N2aaqBEmQjH6keCZGo9vMkokFOti71FwpQO5R67QpjGnuV4xHQMaFZq47FzBDDh+qTCw/mIFghSsTSAppJFX6n00niMQZhQtH7WLjFF
Eh61ls8eZUzY+xghBk4I0jsr00O1Wp4EkJCQqyfPrs7EUE1+qsetO8QEVZEVUg21xdsRGyRu/jRgY9SGEsTGc2EcLdSUMQqrjzUwzpZpJOSf6iGyQgS
eSj0STrvh6FdE8GVocphTpsf8Su3hTGE8U8eCYNNlq5ZNXd625ULcui2upE9QFM0zaJW6H/iynVMbpjCltxgU42q2mTEZSCxEjmkJxGRYHqVZmJrxoB
WUhGUBVTRBUUvhK4lkNkVOIuMuB0k/6RX88I4+GYa6TdXjAfgQ1ZeXT+GPw26f/uWSBrQ+36V9cruwPa8LBtzUbh13ze9pTZUI9eZ7/dNz/cS8frHbf
PPLb5dRPX+sb6JRZ/TFZ/uwG9r2avj82fDs2x9+rZ0a/Hfof3ZRN+P24Avm8Ua3bXcWbmO7d6M1LsXU6elSwzTEeTJgDnDsSXkDruNdUiFl6cucUWVw
sBWbZ+sj9G4NTrn1x3VX3MmCicgZCeZkDB6XyGTCT+NP5DFFqy0vy4ibox3xd3dpIjNjEUdkiYuscDGXlHMyC8VwrwRBG0HUnPWEb5wuxMJLlYIvv1v
8nXv0aOjyzOGWBjcTnoRTBvYZI4ctSnd3dYlmaYhHtFJVmbOc7SVILocfv5i/J4MPPA1SH6lQeomkSpF1gMIK65A8M4chAmOyzM4kqDTW5bZnhcjg23
M43a1pwibGMomSecKWHHaqvhPmRN+WIDMh2nLzZrSP/c9QmS8/Rq2WSFHokRwTK/dh+tTM4tgWyama/9lQ2ob35bi/RneHqJAzE2S+wct7aSe6XtyTD
1PFlrynmdmHYjrNcU9xaKYtShgOM6W4Dp7Gr1zfpLb62vWgUzhS2ksxt+w2PykyXkdxmvhf/Q1u2IIX'
    }
}

## 來產生吧 🎏

為了盡量精簡, 這邊示範的是產生Alpaca格式的資料集, 每筆資料中只有單一輪的問答.  
如果想產生其他的格式, 只要照著這個思路在做擴充即可!

### 系統提示詞, 提示詞範本

透過系統提示詞先跟LLM講好要做哪些事情.

透過題實詞範本來替換不同的文本片段.

In [5]:
system = (
    """你是一個非常有經驗的文章關鍵點提取者,可以精準的從一段文章中找出關鍵重點,並依據這些關鍵點來生成一般人在閱讀這段文章時可能會提出的問題或是想要進一部理解的部份,可以是複合式的問題或各個關鍵點之間的關聯性等, 問題的描述寫的越仔細越好, 你的目標是讓使用者可以在學會這些問題後就能理解整篇文章.
你會依據使用者提供的文章來生成一個資料集,資料集格式如下, 在"Instruction"中填入你設計的問題:
```
[
    {"Instruction": ""},
    {"Instruction": ""},
    {"Instruction": ""},
]
```
你生成的資料集必須盡可能涵蓋整篇文章所提及的內容.
回覆使用者時只需要提供你所生成的資料集,不須做任何額外的說明. 一律使用繁體中文."""
    )

In [6]:
prompt_template =(
    """文章內容:
```
{content}
```
'請依據此文章生成至少 {amount} 筆資料"""
)

### Few-shot prompting

透過 [Few-shot prompting](https://arxiv.org/abs/2205.05638) 的方式預先定好幾輪對話, 可以非常有效的提生LLM按照自己想要的方式來輸出的機率!  

我自己在測試過程中透過Few-shot prompting的方式就連7b, 8b等相對較小都能有非常穩定的輸出.

In [7]:
incontext_user = (
    """文章內容:
```
1. 第一步 - 模型載入：將模型參數移到GPU上。目前的記憶體：模型。
2. 第二步 - 前向傳遞：將輸入通過模型，並存儲中間輸出（激活值）。在這一步驟中，存儲這些激活值佔用了記憶體。雖然存儲所有激活值並不是絕對必要的（參見「梯度檢查點」），但對於反向傳播算法的效率而言，存儲它們實際上是必要的。目前的記憶體：模型 +  激活值。
3. 第三步 - 反向傳播：從網絡末端計算梯度，並在計算的過程中捨棄前向傳遞的激活值。由於我們已經捨棄了前向傳遞的激活值，在反向傳播過程後，記憶體使用量為模型大小的兩倍 - 一份是權重的拷貝，另一份是梯度的拷貝。目前的記憶體：模型 + 梯度。
4. 第四步 - 優化器更新：更新參數，並追蹤優化器的運行參數。許多優化器會追蹤每個模型權重的梯度一階和二階矩的估計值。對於Adam（使用兩個矩），它佔用模型大小的兩倍；對於RMSprop（使用一個矩），它佔用模型大小的一倍；對於SGD（不使用矩），則不佔用模型大小。目前的記憶體：模型 + 梯度 + 梯度矩。
5. 第五步 - 執行下一次迭代：在計算了一次梯度並且優化器進行了一步後，梯度和梯度矩仍然存在。因此，在未來的迭代中，您的總最大記憶體使用量將為：模型 + 激活值 + 梯度 + 梯度矩，這意味著記憶體使用量在第二次迭代時會增加，但之後將保持不變。

現在，對於通常會消耗記憶體的基本了解後，讓我們來看一些特殊情況和優化方法來節省記憶體。
```
請依據此文章生成至少 5 筆資料。"""
)

In [8]:
incontext_assistant = (
    """[
{"Instruction": "什麼是模型載入的第一步驟？"},
{"Instruction": "前向傳遞的過程中，為什麼需要存儲中間輸出（激活值）？"},
{"Instruction": "在反向傳播過程中，為什麼需要捨棄前向傳遞的激活值？"},
{"Instruction": "優化器更新的過程中，什麼是梯度一階和二階矩的估計值？"},
{"Instruction": "在執行下一次迭代時，為什麼梯度和梯度矩仍然存在？"},
{"Instruction": "什麼是梯度檢查點？"},
{"Instruction": "什麼是Adam、RMSprop和SGD優化器的差異？"},
{"Instruction": "如何計算模型的記憶體使用量？"}
]
"""
)

由於每個文本片段的長短不一, 實際應用中可以透過Tokenizer來計算每個文本片段的token數量再乘上一個比例來控制對於該文本片段而言我們要產生幾筆合成資料.

本文為了盡量精簡, 就直接計算文本片段的字數來決定要產生幾筆合成資料 👍️

In [9]:
import re

def count_words(text):
    # 計算中文字符
    chinese_chars = len(re.findall(r'[\u4e00-\u9fff]', text))
    
    # 移除所有中文字符和標點符號，只保留英文和數字
    english_text = re.sub(r'[\u4e00-\u9fff]|[^\w\s]', ' ', text)
    
    # 分割英文單詞並計數
    english_words = len(english_text.split())
    
    # 總字數為中文字符數加上英文單詞數
    total_words = chinese_chars + english_words
    
    return total_words

使用 `pydantic` 來驗證和解析生成的資料

In [10]:
from typing import Optional, Any, Dict
from pydantic import BaseModel, Json, ValidationError

class SyntheticData(BaseModel):
    Instruction: str = ""
    Input: str = ""
    Output: str = ""
    Metadata: Optional[Dict[str, Any]]


class VaildResponse(BaseModel):
    vaild_question: Json

In [11]:
from IPython.display import clear_output
import os
import json
from openai import OpenAI

client = OpenAI(
  base_url = "http://localhost:11434/v1",
  api_key = "api_key"
)

min_gen=3 # 設定每個文本片段最少產生幾筆合成資料
percentage=1 # 設定要產生的合成資料數量是該文本片段總字數的幾％
max_retry=5 # 單一個文本片段產生合成資料的過程中最多重試幾次

gen_fail=[] # 重試次數超過上限仍失敗的話就紀錄起來並跳過
synthetic_dataset=[] # 合成資料集列表

count=0
gen_try=0
for chunk in chunks:    
    amount = max(min_gen, int(count_words(chunk.text) * (percentage/100)))

    prompt = prompt_template.format(content=chunk.text, amount=amount)

    convo = [
        {"role":"system", "content":system},
        {"role":"user", "content":incontext_user},
        {"role":"assistant", "content":incontext_assistant},
        {"role":"user", "content":prompt}
        ]

    while True:
        gen_try += 1

        completion = client.chat.completions.create(
          model="llama3.1:latest",
          messages=convo,
          temperature=0.9,
          top_p=0.7,
          max_tokens=2048,
          stream=False
        )
        try: 
            vaild_response = VaildResponse(vaild_question=completion.choices[0].message.content)
            
            json_data = json.loads(completion.choices[0].message.content)
            #print(f"=== Vaild response ===\n{json_data}\n==========")
            for data in json_data:
              metadata = {'source':md_file, 'content': chunk.text}
              synthetic_data = SyntheticData(Instruction=data['Instruction'], Metadata=metadata)
              synthetic_dataset.append(synthetic_data)

              clear_output(wait=True)
              count+=1
              print(f'Generate instruction {count=}')
            break
        except (ValidationError, json.JSONDecodeError, KeyError) as ex:
          if max_retry < gen_try:
              continue
          else:
              # print(f"=== Invaild response ===\n{completion.choices[0].message.content}\n==========")
              gen_fail.append(completion.choices[0].message.content)
              break

print(synthetic_dataset[:10])

Generate instruction count=81

[
    SyntheticData(
        Instruction='什麼是監督微調（SFT）的局限性？',
        Input='',
        Output='',
        Metadata={
            'source': './example.md',
            'content': 
'然而，監督微調（SFT）也有其局限性。它在利用基礎模型已有的知識時效果最佳。學習完全新的信息，例如未知語言，可能會很
具挑戰性，並導致更頻繁的幻覺。對於基礎模型未知的新領域，建議先在原始數據集上進行持續的預訓練。在另一端，指令模型（
即已經微調過的模型）可能已經非常接近您的需求。例如，一個模型可能表現得非常好，但聲稱它是由OpenAI或Meta訓練的，而不
是您。在這種情況下，您可能希望通過偏好對齊（preference 
alignment）稍微調整指令模型的行為。通過提供選擇和拒絕的樣本（約100到1000個樣本）來微調模型的行為，您可以強迫LLM說明
您訓練了模型，而不是OpenAI。\n\n。⚖️ 監督微調技術（SFT 
Techniques）\n\n目前最受歡迎的三種監督微調技術是完整微調（full fine-tuning）、LoRA和QLoRA。'
        }
    ),
    SyntheticData(
        Instruction='在利用基礎模型已有的知識時，什麼情況下監督微調效果最佳？',
        Input='',
        Output='',
        Metadata={
            'source': './example.md',
            'content': 
'然而，監督微調（SFT）也有其局限性。它在利用基礎模型已有的知識時效果最佳。學習完全新的信息，例如未知語言，可能會很
具挑戰性，並導致更頻繁的幻覺。對於基礎模型未知的新領域，建議先在原始數據集上進行持續的預訓練。在另一端，指令模型（
即已經微調過的模型）可能已經非常接近您的需求。例如，一個模型可能表現得非常好，但聲稱它是由OpenAI或Meta訓練的，而不
是您。在這種情況下，您可能希望通過偏好對齊（preference 
alignment）稍微調整指令模型的行為。通過提供選擇和拒絕的樣本（約100到1000個樣本）來微調模型的行為，您可以強迫LLM說明
您訓練了模型，而不是OpenAI。\n\n。⚖️ 監督微調技術（SFT 
Techniques）\n\n目前最受歡迎的三種監督微調技術是完整微調（full fine-tuning）、LoRA和QLoRA。'
        }
    ),
    SyntheticData(
        Instruction='如何使用偏好對齊（preference alignment）來微調指令模型的行為？',
        Input='',
        Output='',
        Metadata={
            'source': './example.md',
            'content': 
'然而，監督微調（SFT）也有其局限性。它在利用基礎模型已有的知識時效果最佳。學習完全新的信息，例如未知語言，可能會很
具挑戰性，並導致更頻繁的幻覺。對於基礎模型未知的新領域，建議先在原始數據集上進行持續的預訓練。在另一端，指令模型（
即已經微調過的模型）可能已經非常接近您的需求。例如，一個模型可能表現得非常好，但聲稱它是由OpenAI或Meta訓練的，而不
是您。在這種情況下，您可能希望通過偏好對齊（preference 
alignment）稍微調整指令模型的行為。通過提供選擇和拒絕的樣本（約100到1000個樣本）來微調模型的行為，您可以強迫LLM說明
您訓練了模型，而不是OpenAI。\n\n。⚖️ 監督微調技術（SFT 
Techniques）\n\n目前最受歡迎的三種監督微調技術是完整微調（full fine-tuning）、LoRA和QLoRA。'
        }
    ),
    SyntheticData(
        Instruction='什麼是完整微調（Full fine-tuning）？',
        Input='',
        Output='',
        Metadata={
            'source': './example.md',
            'content': '完整微調（Full 
fine-tuning）是最直接的監督微調技術。它涉及使用指令數據集重新訓練預先訓練模型的所有參數。這種方法通常能夠提供最佳結
果，但需要大量的計算資源（需要多個高端GPU來微調一個8B模型）。由於它修改了整個模型，因此也是最具破壞性的方法，可能會
導致之前學習的技能和知識的災難性遺忘。\n\n低秩適應（Low-Rank 
Adaptation，LoRA）是一種流行的參數高效微調技術。與其重新訓練整個模型，LoRA會凍結權重並在每個目標層引入小型適配器（
低秩矩陣）。這使得LoRA能夠訓練的參數數量遠遠低於完整的微調（少於1％），從而減少了內存使用和訓練時間。這種方法是非破
壞性的，因為原始參數被凍結，適配器可以在後續被切換或組合。'
        }
    ),
    SyntheticData(
        Instruction='完整微調的優點和缺點是什麼？',
        Input='',
        Output='',
        Metadata={
            'source': './example.md',
            'content': '完整微調（Full 
fine-tuning）是最直接的監督微調技術。它涉及使用指令數據集重新訓練預先訓練模型的所有參數。這種方法通常能夠提供最佳結
果，但需要大量的計算資源（需要多個高端GPU來微調一個8B模型）。由於它修改了整個模型，因此也是最具破壞性的方法，可能會
導致之前學習的技能和知識的災難性遺忘。\n\n低秩適應（Low-Rank 
Adaptation，LoRA）是一種流行的參數高效微調技術。與其重新訓練整個模型，LoRA會凍結權重並在每個目標層引入小型適配器（
低秩矩陣）。這使得LoRA能夠訓練的參數數量遠遠低於完整的微調（少於1％），從而減少了內存使用和訓練時間。這種方法是非破
壞性的，因為原始參數被凍結，適配器可以在後續被切換或組合。'
        }
    ),
    SyntheticData(
        Instruction='低秩適應（Low-Rank Adaptation，LoRA）的工作原理是什麼？',
        Input='',
        Output='',
        Metadata={
            'source': './example.md',
            'content': '完整微調（Full 
fine-tuning）是最直接的監督微調技術。它涉及使用指令數據集重新訓練預先訓練模型的所有參數。這種方法通常能夠提供最佳結
果，但需要大量的計算資源（需要多個高端GPU來微調一個8B模型）。由於它修改了整個模型，因此也是最具破壞性的方法，可能會
導致之前學習的技能和知識的災難性遺忘。\n\n低秩適應（Low-Rank 
Adaptation，LoRA）是一種流行的參數高效微調技術。與其重新訓練整個模型，LoRA會凍結權重並在每個目標層引入小型適配器（
低秩矩陣）。這使得LoRA能夠訓練的參數數量遠遠低於完整的微調（少於1％），從而減少了內存使用和訓練時間。這種方法是非破
壞性的，因為原始參數被凍結，適配器可以在後續被切換或組合。'
        }
    ),
    SyntheticData(
        Instruction='與完整微調相比，低秩適應的計算資源需求如何？',
        Input='',
        Output='',
        Metadata={
            'source': './example.md',
            'content': '完整微調（Full 
fine-tuning）是最直接的監督微調技術。它涉及使用指令數據集重新訓練預先訓練模型的所有參數。這種方法通常能夠提供最佳結
果，但需要大量的計算資源（需要多個高端GPU來微調一個8B模型）。由於它修改了整個模型，因此也是最具破壞性的方法，可能會
導致之前學習的技能和知識的災難性遺忘。\n\n低秩適應（Low-Rank 
Adaptation，LoRA）是一種流行的參數高效微調技術。與其重新訓練整個模型，LoRA會凍結權重並在每個目標層引入小型適配器（
低秩矩陣）。這使得LoRA能夠訓練的參數數量遠遠低於完整的微調（少於1％），從而減少了內存使用和訓練時間。這種方法是非破
壞性的，因為原始參數被凍結，適配器可以在後續被切換或組合。'
        }
    ),
    SyntheticData(
        Instruction='低秩適應是否是一種破壞性方法？',
        Input=

In [12]:
system = (
    """你是一個輔助閱讀的專家. 使用者會提供一段文章及一個問題給你, 你會先閱讀並理解該文章, 並作為一個專家來回答使用者的問題.
回答的越仔細越好, 如果你無法從文章內容找到問題的答案, 請直接說你無法找到答案, 絕對不要編造任何回答.
回答時必須依據這個格式來提供回答:
```
{"Output": "你的回答"}
```"""
)

In [13]:
prompt_template =(
    """文章內容:
```
{content}
```
問題:
```
{question}
```
注意: 不要使用 "文章中提到...", "依據文章內容...", "根據文章..." 以及所有相關的字眼, 請以一個專家的口吻進行回答."""
)

In [14]:
incontext_user = (
    """文章內容:
```
1. 第一步 - 模型載入：將模型參數移到GPU上。目前的記憶體：模型。
2. 第二步 - 前向傳遞：將輸入通過模型，並存儲中間輸出（激活值）。在這一步驟中，存儲這些激活值佔用了記憶體。雖然存儲所有激活值並不是絕對必要的（參見「梯度檢查點」），但對於反向傳播算法的效率而言，存儲它們實際上是必要的。目前的記憶體：模型 +  激活值。
3. 第三步 - 反向傳播：從網絡末端計算梯度，並在計算的過程中捨棄前向傳遞的激活值。由於我們已經捨棄了前向傳遞的激活值，在反向傳播過程後，記憶體使用量為模型大小的兩倍 - 一份是權重的拷貝，另一份是梯度的拷貝。目前的記憶體：模型 + 梯度。
4. 第四步 - 優化器更新：更新參數，並追蹤優化器的運行參數。許多優化器會追蹤每個模型權重的梯度一階和二階矩的估計值。對於Adam（使用兩個矩），它佔用模型大小的兩倍；對於RMSprop（使用一個矩），它佔用模型大小的一倍；對於SGD（不使用矩），則不佔用模型大小。目前的記憶體：模型 + 梯度 + 梯度矩。
5. 第五步 - 執行下一次迭代：在計算了一次梯度並且優化器進行了一步後，梯度和梯度矩仍然存在。因此，在未來的迭代中，您的總最大記憶體使用量將為：模型 + 激活值 + 梯度 + 梯度矩，這意味著記憶體使用量在第二次迭代時會增加，但之後將保持不變。

現在，對於通常會消耗記憶體的基本了解後，讓我們來看一些特殊情況和優化方法來節省記憶體。
```

問題:
```
優化器更新的過程中，什麼是梯度一階和二階矩的估計值？
```
注意: 不要使用 "文章中提到...", "依據文章內容...", "根據文章..." 以及所有相關的字眼, 請以一個專家的口吻進行回答."""
)

In [15]:
incontext_assistant = (
    """{"Output": "梯度一階和二階矩的估計值是優化器追蹤模型權重的梯度信息，以便在更新模型參數時使用。具體而言，梯度一階矩是指梯度的均值估計，梯度二階矩是指梯度的平方和估計。這些信息可以幫助優化器更好地調整模型參數，尤其是在模型具有高變異性的情況下。"}"""
)

In [16]:
max_retry=10
output_gen_fail=[]

count=0
for data in synthetic_dataset:
    clear_output(wait=True)
    try_gen=0
    prompt = prompt_template.format(content=data.Metadata['content'], question=data.Instruction)

    convo = [
        {"role":"system", "content":system},
        {"role":"user", "content":incontext_user},
        {"role":"assistant", "content":incontext_assistant},
        {"role":"user", "content":prompt}
        ]
    
    while True:
        gen_try += 1

        completion = client.chat.completions.create(
          model="llama3.1:latest",
          messages=convo,
          temperature=0.4,
          top_p=0.7,
          max_tokens=1024,
          stream=False
        )
        try: 
            vaild_response = VaildResponse(vaild_question=completion.choices[0].message.content)
            
            json_data = json.loads(completion.choices[0].message.content)
            # print(f"=== Vaild response ===\n{json_data}\n==========")
            if len(json_data) > 1:
                raise ValueError
            
            count+=1
            print(f'Generate output {count=}')
            data.Output=json_data['Output']
            break
        except (ValidationError, json.JSONDecodeError, KeyError, ValueError) as ex:
          if max_retry < gen_try:
              continue
          else:
              # print(f"=== Invaild response ===\n{completion.choices[0].message.content}\n==========")
              output_gen_fail.append(completion.choices[0].message.content)
              break

print(synthetic_dataset[:10])

Generate output count=81

[
    SyntheticData(
        Instruction='什麼是監督微調（SFT）的局限性？',
        Input='',
        Output='監督微調（SFT）的局限性包括在利用基礎模型已有的知識時效果最佳，但學習完全新的信息，例如未知語言，可
能會很具挑戰性，並導致更頻繁的幻覺。',
        Metadata={
            'source': './example.md',
            'content': 
'然而，監督微調（SFT）也有其局限性。它在利用基礎模型已有的知識時效果最佳。學習完全新的信息，例如未知語言，可能會很
具挑戰性，並導致更頻繁的幻覺。對於基礎模型未知的新領域，建議先在原始數據集上進行持續的預訓練。在另一端，指令模型（
即已經微調過的模型）可能已經非常接近您的需求。例如，一個模型可能表現得非常好，但聲稱它是由OpenAI或Meta訓練的，而不
是您。在這種情況下，您可能希望通過偏好對齊（preference 
alignment）稍微調整指令模型的行為。通過提供選擇和拒絕的樣本（約100到1000個樣本）來微調模型的行為，您可以強迫LLM說明
您訓練了模型，而不是OpenAI。\n\n。⚖️ 監督微調技術（SFT 
Techniques）\n\n目前最受歡迎的三種監督微調技術是完整微調（full fine-tuning）、LoRA和QLoRA。'
        }
    ),
    SyntheticData(
        Instruction='在利用基礎模型已有的知識時，什麼情況下監督微調效果最佳？',
        Input='',
        Output='在利用基礎模型已有的知識時，監督微調效果最佳。',
        Metadata={
            'source': './example.md',
            'content': 
'然而，監督微調（SFT）也有其局限性。它在利用基礎模型已有的知識時效果最佳。學習完全新的信息，例如未知語言，可能會很
具挑戰性，並導致更頻繁的幻覺。對於基礎模型未知的新領域，建議先在原始數據集上進行持續的預訓練。在另一端，指令模型（
即已經微調過的模型）可能已經非常接近您的需求。例如，一個模型可能表現得非常好，但聲稱它是由OpenAI或Meta訓練的，而不
是您。在這種情況下，您可能希望通過偏好對齊（preference 
alignment）稍微調整指令模型的行為。通過提供選擇和拒絕的樣本（約100到1000個樣本）來微調模型的行為，您可以強迫LLM說明
您訓練了模型，而不是OpenAI。\n\n。⚖️ 監督微調技術（SFT 
Techniques）\n\n目前最受歡迎的三種監督微調技術是完整微調（full fine-tuning）、LoRA和QLoRA。'
        }
    ),
    SyntheticData(
        Instruction='如何使用偏好對齊（preference alignment）來微調指令模型的行為？',
        Input='',
        Output='偏好對齊（preference 
alignment）是微調指令模型行為的一種方法。它涉及提供選擇和拒絕的樣本（約100到1000個樣本），以強迫LLM說明您訓練了模型
，而不是OpenAI或其他公司訓練的模型。在這種情況下，您可以通過微調模型行為，讓它更符合您的偏好和需求。',
        Metadata={
            'source': './example.md',
            'content': 
'然而，監督微調（SFT）也有其局限性。它在利用基礎模型已有的知識時效果最佳。學習完全新的信息，例如未知語言，可能會很
具挑戰性，並導致更頻繁的幻覺。對於基礎模型未知的新領域，建議先在原始數據集上進行持續的預訓練。在另一端，指令模型（
即已經微調過的模型）可能已經非常接近您的需求。例如，一個模型可能表現得非常好，但聲稱它是由OpenAI或Meta訓練的，而不
是您。在這種情況下，您可能希望通過偏好對齊（preference 
alignment）稍微調整指令模型的行為。通過提供選擇和拒絕的樣本（約100到1000個樣本）來微調模型的行為，您可以強迫LLM說明
您訓練了模型，而不是OpenAI。\n\n。⚖️ 監督微調技術（SFT 
Techniques）\n\n目前最受歡迎的三種監督微調技術是完整微調（full fine-tuning）、LoRA和QLoRA。'
        }
    ),
    SyntheticData(
        Instruction='什麼是完整微調（Full fine-tuning）？',
        Input='',
        Output='完整微調是監督微調的一種方法，涉及使用指令數據集重新訓練預先訓練模型的所有參數。',
        Metadata={
            'source': './example.md',
            'content': '完整微調（Full 
fine-tuning）是最直接的監督微調技術。它涉及使用指令數據集重新訓練預先訓練模型的所有參數。這種方法通常能夠提供最佳結
果，但需要大量的計算資源（需要多個高端GPU來微調一個8B模型）。由於它修改了整個模型，因此也是最具破壞性的方法，可能會
導致之前學習的技能和知識的災難性遺忘。\n\n低秩適應（Low-Rank 
Adaptation，LoRA）是一種流行的參數高效微調技術。與其重新訓練整個模型，LoRA會凍結權重並在每個目標層引入小型適配器（
低秩矩陣）。這使得LoRA能夠訓練的參數數量遠遠低於完整的微調（少於1％），從而減少了內存使用和訓練時間。這種方法是非破
壞性的，因為原始參數被凍結，適配器可以在後續被切換或組合。'
        }
    ),
    SyntheticData(
        Instruction='完整微調的優點和缺點是什麼？',
        Input='',
        Output='完整微調的優點是可以提供最佳結果，但缺點是需要大量的計算資源和修改了整個模型，因此也是最具破壞性的
方法，可能會導致之前學習的技能和知識的災難性遺忘。',
        Metadata={
            'source': './example.md',
            'content': '完整微調（Full 
fine-tuning）是最直接的監督微調技術。它涉及使用指令數據集重新訓練預先訓練模型的所有參數。這種方法通常能夠提供最佳結
果，但需要大量的計算資源（需要多個高端GPU來微調一個8B模型）。由於它修改了整個模型，因此也是最具破壞性的方法，可能會
導致之前學習的技能和知識的災難性遺忘。\n\n低秩適應（Low-Rank 
Adaptation，LoRA）是一種流行的參數高效微調技術。與其重新訓練整個模型，LoRA會凍結權重並在每個目標層引入小型適配器（
低秩矩陣）。這使得LoRA能夠訓練的參數數量遠遠低於完整的微調（少於1％），從而減少了內存使用和訓練時間。這種方法是非破
壞性的，因為原始參數被凍結，適配器可以在後續被切換或組合。'
        }
    ),
    SyntheticData(
        Instruction='低秩適應（Low-Rank Adaptation，LoRA）的工作原理是什麼？',
        Input='',
        Output='低秩適應（Low-Rank 
Adaptation，LoRA）是通過在每個目標層引入小型適配器（低秩矩陣）來微調預先訓練模型的參數。這些適配器可以幫助模型適應
新的資料集，而不改變原有的模型參數。這種方法是非破壞性的，因為它們只添加了少量新參數，沒有修改原始的模型權重。',
        Metadata={
            'source': './example.md',
            'content': '完整微調（Full 
fine-tuning）是最直接的監督微調技術。它涉及使用指令數據集重新訓練預先訓練模型的所有參數。這種方法通常能夠提供最佳結
果，但需要大量的計算資源（需要多個高端GPU來微調一個8B模型）。由於它修改了整個模型，因此也是最具破壞性的方法，可能會
導致之前學習的技能和知識的災難性遺忘。\n\n低秩適應（Low-Rank 
Adaptation，LoRA）是一種流行的參數高效微調技術。與其重新訓練整個模型，LoRA會凍結權重並在每個目標層引入小型適配器（
低秩矩陣）。這使得LoRA能夠訓練的參數數量遠遠低於完整的微調（少於1％），從而減少了內存使用和訓練時間。這種方法是非破
壞性的，因為原始參數被凍結，適配器可以在後續被切換或組合。'
        }
    ),
    SyntheticData(
        Instruction='與完整微調相比，低秩適應的計算資源需求如何？',
        Input='',
        Output='低秩適應比完整微調需要少於1％的計算資源，主要是因

完成後可以存成 `Dataset`格式用來分享到HuggingFace, 也可以存成 `Dataframe` 做其他處理

In [17]:
import os
from datetime import datetime
from datasets import Dataset
import pandas as pd

data_dict_list = [data.model_dump() for data in synthetic_dataset]

dataset = Dataset.from_list(data_dict_list)

# save_dir = f"{datetime.now():%Y%m%d%H%M%S}"
# os.makedirs(save_dir, exist_ok=True)
# dataset.save_to_disk(save_dir)

df = pd.DataFrame(data_dict_list)

# csv_file=os.path.join(save_dir, f"{save_dir}_SyntheticDataset.csv")
# df.to_csv(csv_file, index=False)

In [18]:
df[:10]

,Instruction,Input,Output,Metadata
0,什麼是監督微調（SFT）的局限性？,,監督微調（SFT）的局限性包括在利用基礎模型已有的知識時效果最佳，但學習完全新的信息，例如未...,"{'source': './example.md', 'content': '然而，監督微調..."
1,在利用基礎模型已有的知識時，什麼情況下監督微調效果最佳？,,在利用基礎模型已有的知識時，監督微調效果最佳。,"{'source': './example.md', 'content': '然而，監督微調..."
2,如何使用偏好對齊（preference alignment）來微調指令模型的行為？,,偏好對齊（preference alignment）是微調指令模型行為的一種方法。它涉及提供...,"{'source': './example.md', 'content': '然而，監督微調..."
3,什麼是完整微調（Full fine-tuning）？,,完整微調是監督微調的一種方法，涉及使用指令數據集重新訓練預先訓練模型的所有參數。,"{'source': './example.md', 'content': '完整微調（Fu..."
4,完整微調的優點和缺點是什麼？,,完整微調的優點是可以提供最佳結果，但缺點是需要大量的計算資源和修改了整個模型，因此也是最具破...,"{'source': './example.md', 'content': '完整微調（Fu..."
5,低秩適應（Low-Rank Adaptation，LoRA）的工作原理是什麼？,,低秩適應（Low-Rank Adaptation，LoRA）是通過在每個目標層引入小型適配器...,"{'source': './example.md', 'content': '完整微調（Fu..."
6,與完整微調相比，低秩適應的計算資源需求如何？,,低秩適應比完整微調需要少於1％的計算資源，主要是因為它只在每個目標層引入小型適配器，而不是重...,"{'source': './example.md', 'content': '完整微調（Fu..."
7,低秩適應是否是一種破壞性方法？,,否，低秩適應是一種非破壞性方法。,"{'source': './example.md', 'content': '完整微調（Fu..."
8,完整微調和低秩適應哪種方法能夠提供最佳結果？,,完整微調通常能夠提供最佳結果。,"{'source': './example.md', 'content': '完整微調（Fu..."
9,什麼是QLoRA，它提供了哪些優勢？,,QLoRA是LoRA的一個擴展，提供了更大的記憶體節省。它可以在標準LoRA的基礎上節省高達...,"{'source': './example.md', 'content': 'QLoRA（量..."


## Next step

資料集生成了, 接下來可以做什麼？

- 這些資料集真的夠好嗎？  
  [Textbooks Are All You Need](https://arxiv.org/abs/2306.11644) 中提到高精度的資料對模型的成效是有很關鍵的影響的.  
  可以嘗試用 Nvidia 推出的 [nemotron-4-340b-instruct](https://build.nvidia.com/nvidia/nemotron-4-340b-instruct) 來篩選出優質的資料, 但是目前並未對中文(尤其是繁體中文)做過優化, 成效可能不佳.  
  也可以試看看開源的 [internlm/internlm2-20b-reward](https://huggingface.co/internlm/internlm2-20b-reward), 是個對中文有做過優化的Reward模型.

- 調整提示詞看看會發生什麼事  
  本文使用到的提示詞都是我自己測試時使用的提示詞, 我在測試過程中發現可能只加一兩個字就會得到非常不同的結果！  
  例如:
  - prompt: "...依據文章產生一個問題", LLM: "問題：什麼是梯度檢查點，為什麼存儲所有激活值不是絕對必要的？"
  - prompt: "...依據文章產生一個讓人想笑的問題", LLM: 問題："如果模型在跑了5步後突然開始思考自己的存在意義，是否會出現"存在危機"，導致記憶體使用量增加，然後需要進行優化更新？😂"  (emoji也是LLM給的)
  
- 換個模型試試吧！  
  不同的模型產生出來的結果都不一樣, 有趣的同時還可以增加資料量！

- Fine-tune你專屬的模型吧！  
  本文中使用到的示範文件就是我自己<s>丟給AI</s>將這篇[Fine-tune Llama 3.1 Ultra-Efficiently with Unsloth](https://huggingface.co/blog/mlabonne/sft-llama3)翻成中文的, 文中說明了如何使用Unsloth微調LLM.

- 插一手  
  藉由 [Human in the loop(HITL)](https://cloud.google.com/discover/human-in-the-loop)的方式, 人類跟AI協作, 在數據生成的過程中加入人類的輔助, 提升數據品質!

- 優化程式碼  
  本文中提供的程式碼都盡量精簡, 還有很多地方可以在優化, 做更細節的處理, 靠你了🫠